# Using Custom Transformers in Pyreal

In this tutorial, we will be using Pyreal to investigate the California Housing Dataset.

In order to generate useful explanations, we will making a few custom transformers, with functionality specific to this use-case.

## Data Loading

This dataset includes 9 predictor variables, and one target variable. Each row in the dataset refers to a block of houses in California. The target variable is the median house value in this block.

**Run the cell below to load in the California Housing Dataset.**

In [1]:
import matplotlib.pyplot as plt
from urllib.parse import urljoin
import pandas as pd

AWS_BASE_URL = 'https://pyreal-data.s3.amazonaws.com/'
data_url = urljoin(AWS_BASE_URL, "usability_study/california.csv")
data = pd.read_csv(data_url)

city_url = urljoin(AWS_BASE_URL, "usability_study/cal_cities_lat_long.csv")
cities = pd.read_csv(city_url)

data = data[data["median_house_value"] < 500000]

data = data.sample(5000, random_state=100)  # we will work with a truncated dataset to avoid memory crashes

X = data.drop("median_house_value", axis=1)
y = data["median_house_value"]

data.sample(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,median_house_value
338,-122.18,37.74,46,2103,391.0,1339,354,2.2467,NEAR BAY,88900
1565,-121.93,37.78,2,227,35.0,114,49,3.1591,<1H OCEAN,434700
7601,-118.26,33.89,36,2230,417.0,1395,381,2.8493,<1H OCEAN,109600
5110,-118.31,33.96,46,1686,303.0,870,320,3.4643,<1H OCEAN,136300
678,-122.15,37.69,36,1545,273.0,863,267,4.0109,NEAR BAY,192900
4226,-118.29,34.10,39,2196,582.0,1165,538,2.9417,<1H OCEAN,254200
10941,-117.88,33.74,29,720,174.0,1045,181,3.1964,<1H OCEAN,151900
18797,-122.45,40.85,20,2701,573.0,892,358,2.7736,INLAND,107800
16931,-122.31,37.57,42,3157,676.0,1603,629,3.7422,NEAR OCEAN,292600
406,-122.27,37.90,52,2041,270.0,671,253,6.9414,NEAR BAY,417500


We will be working with a pretrained model to predict the median house values for each block.

**Run the code below to load in the model.**

In [2]:
import lightgbm
import requests

model_url = urljoin(AWS_BASE_URL, "usability_study/model.model")
r = requests.get(model_url, allow_redirects=True)
open(urljoin('data', 'model.model'), 'wb').write(r.content)

model = lightgbm.Booster(model_file=urljoin('data', 'model.model'))

## Custom Transformers Basics

Pyreal generates ML explanations using Explainer objects, which take in data Transformers through their `transformer` parameter. These Transformers take in three flags in their initialization, two of which we will use in this tutorial:

- Transformers with a `model=True` flag take data from the original feature space (as we loaded in above) to the feature space used by the model.
- Transformers with an `interpret=True` flag take data from the original feature space to a feature space more readable or interpretable by humans.

<div class="alert alert-block alert-info" style="background-color: white; border: 2px solid; padding: 10px">
    <b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Note</b><br>
    <p style="color: black">
         For information about the third transformer flag (<code>algorithm</code>), please see the <a href="https://github.com/sibyl-dev/pyreal/blob/dev/tutorials/advanced_explanation_generation.ipynb">advanced explanation generation tutorial.</a>
    </p>
</div>

The Pyreal `transformer` module has some common transformers available for use, but some use-cases may require you to write your own transformer. This can be done by extending the base `Transformer` class.

### Transformer Functions
When defining a custom transformer, you will need to consider three types of functions:
- `data_transform` (*required*): A single function that transforms the data from space A to B.
- `inverse_transform_explanation_XXX` (*optional*): Functions that transform an explanation from space B to A. This type of function only needs to be considered if the transformer is used by the explanation algorithm and leads to the data being more obfuscated/less interpretable (ie, will have the `algorithm` flag set to True and the `interpret` flag set to False)
- `transform_explanation_XXX` (*optional*): Functions that transform an explanation from space A to B. This type of function only needs to be considered if the transformer is used to make data and explanations more interpretable than the algorithm-ready state (ie, will have the `algorithm` flag set to False and the `interpret` flag set to True)

The `transform_explantion` type functions are written per Explanation output type. For this tutorial, we will consider additive local feature contribution and additive global feature importance explanations. At the end of this tutorial, we will consider some special cases.

### Custom Transformer Example 1: Per-Household Averager

Let's take a look at one possible custom transformer we can add, which will average the values of certain features per household. We will follow these steps to write the function:

1. Define the transformer `__init__()` method, using a `super()` call for the parent `Transformer` class. The function can take optional arguments to configure the transformer. We will take in a list of columns to average.
2. Define the `data_transform()` function, which takes an input DataFrame `x` and returns `x` after undergoing the transformation. In this case, we simply divide the selected columns by the households feature.
3. Consider which flags we expect to be used with this transformer. In this case, our transformation will be used for the explanation algorithm, but also makes the data more interpretable, so our flags are `interpret=True` and `algorithm=True`. Therefore, we do not need to define any explanation transform functions for this use case.

**Run the cell below to define the PerHouseholdAverager**


In [5]:
from pyreal.transformers import OneHotEncoder, MultiTypeImputer, Transformer, fit_transformers, run_transformers
from pyreal.types.explanations.feature_based import AdditiveFeatureContributionExplanation

class PerHouseholdAverager(Transformer):
    def __init__(self, columns, **kwargs):
        # columns: the columns to average. Must be list of strings (column names)
        self.columns = columns
        super().__init__(**kwargs)

    def data_transform(self, x):
        # Transform the data by adding a new column from total_[column] called
        #   average_[column]. This feature represents the average value of
        #   [column] per household.
        for column in self.columns:
            name = column.replace("total", "average")
            x[name] = x[column] / x["households"]
        return x

### Custom Transformer Example 2: City Converter

Now, let's take a look at another custom transformer, this one requiring an explanation transform. This transformer will be used to convert latitude/longitude values into city areas, based on the closest city to the given coordinates.

Again, we will follow the following steps:

1. Define the transformer `__init__()` method, using a `super()` call for the parent `Transformer` class. The function can take optional arguments to configure the transformer. In this case, we will not take in any parameters.
2. Define the `data_transform()` function, which takes an input DataFrame `x` and returns `x` after undergoing the transformation. In this case, we convert long/lat values to nearby cities.
3. Consider which flags we expect to be used with this transformer. In this case, our transformation will be used to make the data more interpretable, but will not be fed into the model, so our flags are `interpret=True` and `algorithm=model=False`. Therefore, the `transform_explanation_XXX` functions will be called, but not the `inverse_transform_explanation` functions.
4. Consider the explanation output types you are interested in. For now, let's consider additive local feature contribution and additive global feature importance explanations. These are explanations that provide an importance weighting to each feature, and where these importances can be meaningfully added together to represent the importance of multiple features. We will need to define `transform_explanation_additive_feature_contribution` and `transform_explanation_additive_feature_importance` functions.


<div class="alert alert-block alert-info" style="background-color: white; border: 2px solid; padding: 10px">
    <b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Note</b><br>
    <p style="color: black">
         Note that explanation output types are heirarchial, and Pyreal will use the explanation transforms of higher-level types when the lower-level types are not available. For example, if <code>transform_explanation_additive_feature_contribution</code> is not defined, Pyreal will attempt to use <code>transform_explanation_feature_contribution</code>.
    </p>
</div>


In [6]:
class CityConverter(Transformer):
    def __init__(self, **kwargs):
        self.cities = cities
        super().__init__(**kwargs)

    def data_transform(self, x):
        # Converts latitude/longitude coordinates to closest city name. Note that
        #    we are using a very rough estimate here, assuming constant size.
        flag = False
        if isinstance(x, pd.Series):
            x = x.to_frame().T
            flag = True
        for index, row in self.cities.iterrows():
            lat = row["Latitude"]
            lon = row["Longitude"]
            x.loc[(x["latitude"] > lat-0.1) & (x["latitude"] < lat+0.1) & (x["longitude"] > lon-0.1) & (x["longitude"] < lon+0.1), "city"] = row["Name"]
        x = x.drop("latitude", axis=1)
        x = x.drop("longitude", axis=1)
        if flag:
            x = x.squeeze(axis=0)
        return x

    def transform_explanation_additive_feature_contribution(self, explanation):
        df = explanation.get()  # A DataFrame with one row per instance and one column per feature
        return AdditiveFeatureContributionExplanation(helper_sum_lat_long_columns(df))

    def transform_explanation_additive_feature_importance(self, explanation):
        df = explanation.get()  # A DataFrame with one row and one column per feature
        return AdditiveFeatureImportanceExplanation(helper_sum_lat_long_columns(df))

    def helper_sum_lat_long_columns(explanation):
        # In the case of additive contributions or importances, we can combine the latitude and
        #    longitude explanation contributions by summing to get the city contribution. In this
        #    case, our implementation is almost identical for both types of explanations
        explanation["city"] = explanation["longitude"] + explanation["latitude"]
        explanation = explanation.drop("longitude", axis=1)
        explanation = explanation.drop("latitude", axis=1)
        return explanation

## Using Transformers

In the next code cell, we will using our custom defined transformers, as well as some predefined Pyreal transformers, to generate local and global explanations.

In [9]:
from pyreal.explainers import LocalFeatureContribution, GlobalFeatureImportance
from pyreal.transformers import OneHotEncoder, MultiTypeImputer, fit_transformers

# Initialize and fit transformers using fit_transformers
transformers = [MultiTypeImputer(model=True),
                OneHotEncoder(columns="ocean_proximity", model=True),
                PerHouseholdAverager(columns=["total_bedrooms", "total_rooms"], model=True, interpret=True),
                CityConverter(model=False, interpret=True)]
fit_transformers(transformers, X)

# Initialize and fit the explainers
local_explainer = LocalFeatureContribution(model=model, x_train_orig=X, transformers=transformers, fit_on_init=True)
global_explainer = GlobalFeatureImportance(model=model, x_train_orig=X, transformers=transformers, fit_on_init=True)

In [10]:
from pyreal.utils import visualize

# Generate and visualize the explanation
local_explanation, x_interpret = local_explainer.produce(X.iloc[0])
#global_explanation = global_explainer.produce()

print(local_explanation, x_interpret)

visualize.plot_top_contributors(local_explanation, values=x_interpret, select_by="absolute", show=True)
visualize.plot_top_contributors(global_explanation, select_by="absolute", show=True)

NameError: name 'helper_sum_lat_long_columns' is not defined

# Generating Global Explanations

You can now start using Pyreal Explainers to investigate the model. Remember to take a look at the links at the top of this page as needed.

We will begin by generating a *global* explanation, or an explanation of how the model makes predictions in general.

⭐**In the next cell, initialize, fit, and call the produce function on a `GlobalFeatureImportance` Explainer.**

⭐**Remember, you will need to begin by initalizing and fitting the required transformers. Revisit the cells above for details on the transformers you will need and their flags. You can use the `fit_transformers` function to quickly fit them.**

In [ ]:
from pyreal.explainers import GlobalFeatureImportance
from pyreal.transformers import fit_transformers
from pyreal.utils import visualize

# Step one: Initialize and fit transformers using fit_transformers
# ---- Your code here ----

# Step two: Initialize and fit the explainer
# ---- Your code here ----

# Step three: Generate and visualize the explanation
# ---- Your code here ----

⭐**Now, please press the "next" button on the Qualtrics survey tab and answer the next set of questions, referencing this tab as needed.** ⭐




# Generating Local Explanations

⭐ **Now, please consider the `sample_block` listed below, which refers to a hypothetical block of houses that might exist in California. Run the code block.**



In [ ]:
sample_block = pd.Series({
  "longitude": -122.23,
  "latitude": 37.88,
  "housing_median_age": 16,
  "total_rooms": 672,
  "total_bedrooms": 230,
  "population": 220,
  "households": 52,
  "median_income":  5.3252,
  "ocean_proximity": "NEAR BAY"
})


We will now generate a *local* explanation, or an explanation of why the model makes the prediction it does for the specific `sample_block` above.

⭐**In the next cell, initialize, fit, and call the produce function on a `LocalFeatureContribution` Explainer. Generate an explanation for the prediction of the `sample_block` above.**

⭐**You can likely reuse the already-fit transformers from the previous section. If you would like to use any more, remember to fit them before using.**

In [ ]:
from pyreal.explainers import LocalFeatureContribution
from pyreal.transformers import fit_transformers
from pyreal.utils import visualize

# Step one: Initialize and fit the explainer
# ---- Your code here ----

# Step two: Generate and visualize the explanation
# ---- Your code here ----

⭐**Now, please press the "next" button on the Qualtrics survey tab and answer the next set of questions, referencing this tab as needed.** ⭐


# Downloading This Notebook

⭐**Once you have finished answering all questions, save and download this notebook to a location of your choosing using `File` &#8594; `Download` &#8594; `Download .ipynb`, in the upper left toolbar of this page. You will need to upload it shortly to the Qualtrics survey.**

⭐**Please return to the Qualtrics survey now and press next, then follow instructions to upload this notebook, and then answer final reflection questions.**
